In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from shapely.geometry import Point
import fiona

print("Loaded libraries successfully.")

Loaded libraries successfully.


In [3]:
# take in paths for files (gedi and prodes)
gedi = input("Enter path to GEDI file (.csv): ")
prodes = input("Enter path to PRODES file (.gpkg): ")

lat_col, lon_col = 'latitude', 'longitude'  # specify the column names for latitude and longitude

gedi_df = pd.read_csv(gedi)
prodes_df = gpd.read_file(prodes)

/Users/akshatk/miniconda3/lib/python3.13/site-packages/pyogrio/geopandas.py:275: UserWarning: More than one layer found in 'prodes_amazonia_legal.gpkg': 'accumulated_deforestation_2007' (default), 'hydrography', 'yearly_deforestation', 'no_forest', 'residual'. Specify layer parameter to avoid this warning.
  result = read_func(


In [4]:
# print all states in prodes_df
print("States in PRODES data:", prodes_df['state'].unique()) 

# get number of rows for each state
state_counts = prodes_df['state'].value_counts()
print("Number of rows for each state in PRODES data:\n", state_counts)

# chose "AC" (Acre) as state to test histograms on since it had 24k rows of data
acre_prodes = prodes_df[prodes_df['state'] == 'AC']

States in PRODES data: ['RR' 'PA' 'AM' 'MA' 'MT' 'TO' 'AC' 'RO' 'AP']
Number of rows for each state in PRODES data:
 state
PA    66352
AM    45237
MT    25989
AC    24107
MA    17015
RO    11623
AP     6662
RR     6644
TO     2781
Name: count, dtype: int64


In [5]:
layers = fiona.listlayers(prodes)
print(f"Available layers in GPKG: {layers}")

Available layers in GPKG: ['accumulated_deforestation_2007', 'hydrography', 'yearly_deforestation', 'no_forest', 'residual']


In [6]:
for layer in layers:
    print(f"\nReading '{layer}':")
    with fiona.open(prodes, layer=layer) as lyr:
        print(lyr.schema)
        print(lyr.crs)


Reading 'accumulated_deforestation_2007':
{'properties': {'state': 'str:99', 'path_row': 'str:20', 'main_class': 'str:255', 'class_name': 'str:255', 'def_cloud': 'float', 'julian_day': 'int32', 'image_date': 'date', 'year': 'float', 'area_km': 'float', 'scene_id': 'int32', 'source': 'str:50', 'satellite': 'str:50', 'sensor': 'str:50', 'uuid': 'str'}, 'geometry': 'Unknown'}
EPSG:4674

Reading 'hydrography':
{'properties': {'state': 'str:99', 'path_row': 'str:20', 'main_class': 'str:255', 'class_name': 'str:255', 'def_cloud': 'float', 'julian_day': 'int32', 'image_date': 'date', 'year': 'float', 'area_km': 'float', 'scene_id': 'int32', 'source': 'str:50', 'satellite': 'str:50', 'sensor': 'str:50', 'uuid': 'str'}, 'geometry': 'Unknown'}
EPSG:4674

Reading 'yearly_deforestation':
{'properties': {'state': 'str:99', 'path_row': 'str:20', 'main_class': 'str:255', 'class_name': 'str:255', 'sub_class': 'str:255', 'def_cloud': 'float', 'julian_day': 'int32', 'image_date': 'date', 'year': 'float

In [7]:
# Convert GEDI points into GeoDataFrame for spatial operations
gedi_gdf = gpd.GeoDataFrame(
    data = gedi_df,
    geometry = gpd.points_from_xy(gedi_df.longitude, gedi_df.latitude),
    crs = "EPSG:4674"  # CHANGE THIS BASED ON GEDI DATA CRS (above)
)

# Ensure both GeoDataFrames use the same CRS
if gedi_gdf.crs != acre_prodes.crs:
    print(f"The CRS of the GEDI data {gedi_gdf.crs} does not match the PRODES data CRS {acre_prodes.crs}")
    gedi_gdf = gedi_gdf.to_crs(acre_prodes.crs)
    print(f"Transformed GEDI data to CRS of PRODES data: {gedi_gdf.crs}")


In [8]:
# First few rows of new gedi GeoDataFrame (converted from CSV)
gedi_gdf.head()

,source_url,beam,shot_number,latitude,longitude,rh25,rh50,rh75,rh95,sensitivity,solar_elevation,geometry
0,https://harmony.earthdata.nasa.gov/service-res...,BEAM0101,338350500100046529,-7.443723,-73.928047,10.03,19.99,22.31,24.78,0.972721,35.823444,POINT (-73.92805 -7.44372)
1,https://harmony.earthdata.nasa.gov/service-res...,BEAM0101,338350500100046530,-7.443301,-73.927743,13.25,20.02,23.02,25.98,0.978348,35.823600,POINT (-73.92774 -7.4433)
2,https://harmony.earthdata.nasa.gov/service-res...,BEAM0101,338350500100046531,-7.442881,-73.927440,4.26,9.06,14.52,20.47,0.973001,35.823753,POINT (-73.92744 -7.44288)
3,https://harmony.earthdata.nasa.gov/service-res...,BEAM0101,338350500100046532,-7.442461,-73.927137,6.70,10.48,19.31,30.69,0.976818,35.823906,POINT (-73.92714 -7.44246)
4,https://harmony.earthdata.nasa.gov/service-res...,BEAM0101,338350500100046533,-7.442040,-73.926834,6.40,13.02,17.25,26.01,0.946995,35.824060,POINT (-73.92683 -7.44204)


In [9]:
# First few rows of Acre PRODES data
acre_prodes.head()

,state,path_row,main_class,class_name,def_cloud,julian_day,image_date,year,area_km,scene_id,source,satellite,sensor,uuid,geometry
46,AC,00566,DESMATAMENTO,d2007,0.0,0.0,NaT,2007.0,0.000057,0.0,None,None,None,f5b6c3f0-c713-418f-969c-3d82ad96747f,"MULTIPOLYGON (((-72.40155 -8.94368, -72.40151 ..."
64,AC,00665,DESMATAMENTO,d2007,0.0,0.0,NaT,2007.0,23.322735,0.0,None,None,None,77f4d1db-27d3-49c6-9d74-0e65d2274766,"MULTIPOLYGON (((-73.63409 -7.46974, -73.63409 ..."
104,AC,00267,DESMATAMENTO,d2007,0.0,0.0,NaT,2007.0,0.000092,0.0,None,None,None,060e7351-5b30-433c-988d-18ffa3a52032,"MULTIPOLYGON (((-68.43083 -9.49863, -68.4308 -..."
107,AC,00466,DESMATAMENTO,d2007,0.0,0.0,NaT,2007.0,0.007055,0.0,None,None,None,2f40356d-3bf6-4c88-9d0e-0b3f5408361c,"MULTIPOLYGON (((-70.89457 -8.19909, -70.89457 ..."
108,AC,00566,DESMATAMENTO,d2007,0.0,0.0,NaT,2007.0,0.000639,0.0,None,None,None,8174cbea-5625-47ce-92a8-06fda7e9e2ef,"MULTIPOLYGON (((-72.74963 -8.30158, -72.74963 ..."


In [10]:
# Match GEDI points to PRODES polygons
matched_gdf = acre_prodes.sjoin(
    gedi_gdf,
    how="inner",
    predicate="contains" # checks that the prodes polygon contains the gedi point in it
)

In [ ]:
# Get some statistics
print("Total number of GEDI points in Acre:", len(gedi_gdf))
print("Total number of PRODES polygons in Acre:", len(acre_prodes))
print("Total number of matched pairs (aka rows in matched_gdf):", len(matched_gdf))
print(f"Columns in the matched dataset: \n{matched_gdf.columns.tolist()}")

Total number of GEDI points in Acre: 118491
Total number of PRODES polygons in Acre: 24107
Total number of matched pairs (aka rows in matched_gdf): 5834
Columns in the matched dataset: 
['state', 'path_row', 'main_class', 'class_name', 'def_cloud', 'julian_day', 'image_date', 'year', 'area_km', 'scene_id', 'source', 'satellite', 'sensor', 'uuid', 'geometry', 'index_right', 'source_url', 'beam', 'shot_number', 'latitude', 'longitude', 'rh25', 'rh50', 'rh75', 'rh95', 'sensitivity', 'solar_elevation']
First few rows of matched data:
    state path_row    main_class class_name  def_cloud  julian_day image_date  \
549    AC    00267  DESMATAMENTO      d2007        0.0         0.0        NaT   
549    AC    00267  DESMATAMENTO      d2007        0.0         0.0        NaT   
549    AC    00267  DESMATAMENTO      d2007        0.0         0.0        NaT   
549    AC    00267  DESMATAMENTO      d2007        0.0         0.0        NaT   
549    AC    00267  DESMATAMENTO      d2007        0.0     

In [19]:
# Print head of matched data

print(matched_gdf['main_class'].value_counts())


main_class
DESMATAMENTO    5834
Name: count, dtype: int64
